# Prerequisites

In [4]:
import numpy as np
import matplotlib.pyplot as pl
import numpy as np
import scanpy.api as sc
import pandas as pd
from scanpy.tools import rna_velocity
from anndata import AnnData
import seaborn as sns
from scipy.sparse import csr_matrix
import networkx as nx
import xlsxwriter
from matplotlib import rcParams
import seaborn as sns
import scipy as sci
import gseapy as gp
sc.settings.verbosity = 3
sc.logging.print_versions()

scanpy==1.3.2 anndata==0.6.10 numpy==1.15.4 scipy==1.2.1 pandas==0.23.4 scikit-learn==0.20.0 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


# Load raw data

In [ ]:
# Read cellranger files for all four samples
filename = './E12_5_counts/mm10/matrix.mtx'
filename_genes = './E12_5_counts/mm10/genes.tsv'
filename_barcodes = './E12_5_counts/mm10/barcodes.tsv'

e125 = sc.read(filename).transpose()
e125.var_names = np.genfromtxt(filename_genes, dtype=str)[:, 1]
e125.obs_names = np.genfromtxt(filename_barcodes, dtype=str)

filename = './E13_5_counts/mm10/matrix.mtx'
filename_genes = './E13_5_counts/mm10/genes.tsv'
filename_barcodes = './E13_5_counts/mm10/barcodes.tsv'

e135 = sc.read(filename).transpose()
e135.var_names = np.genfromtxt(filename_genes, dtype=str)[:, 1]
e135.obs_names = np.genfromtxt(filename_barcodes, dtype=str)

filename = './E14_5_counts/mm10/matrix.mtx'
filename_genes = './E14_5_counts/mm10/genes.tsv'
filename_barcodes = './E14_5_counts/mm10/barcodes.tsv'

e145 = sc.read(filename).transpose()
e145.var_names = np.genfromtxt(filename_genes, dtype=str)[:, 1]
e145.obs_names = np.genfromtxt(filename_barcodes, dtype=str)

filename = './E15_5_counts/mm10/matrix.mtx'
filename_genes = './E15_5_counts/mm10/genes.tsv'
filename_barcodes = './E15_5_counts/mm10/barcodes.tsv'

e155 = sc.read(filename).transpose()
e155.var_names = np.genfromtxt(filename_genes, dtype=str)[:, 1]
e155.obs_names = np.genfromtxt(filename_barcodes, dtype=str)




# Add dev. timepoint label for each sample
e125.obs['day'] = '12.5'
e135.obs['day'] = '13.5'
e145.obs['day'] = '14.5'
e155.obs['day'] = '15.5'
# Create Concatenated anndata object for all timepoints
alldays = e125.concatenate(e135, e145, e155)
# Deleting individual day arrays
del e125
del e135
del e145
del e155

# Preprocessing

## QC

Quality control - calculate QC covariates for all anndata objects

In [ ]:
print(alldays.obs['day'].value_counts()) # number of cells per sample (day)

In [ ]:
# #counts per cell
alldays.obs['n_counts'] = alldays.X.sum(1)
# #logcounts per cell
alldays.obs['log_counts'] = np.log(alldays.obs['n_counts'])
# #genes per cell
alldays.obs['n_genes'] = (alldays.X > 0).sum(1)
# mitochondrial gene fraction
mt_gene_mask = [gene.startswith('mt-') for gene in alldays.var_names]
mt_gene_index = np.where(mt_gene_mask)[0]
alldays.obs['mt_frac'] = alldays.X[:,mt_gene_index].sum(1) / alldays.X.sum(1)

Quality control - plot QC metrics

In [ ]:
#Sample quality plots
sc.pl.violin(alldays, ['n_counts', 'mt_frac'], groupby='day', size=1, log=False, cut=0)
#sc.pl.violin(alldays, 'mt_frac', groupby='day')

Filter cells according to identified QC thresholds

In [ ]:
print('Total number of cells: {:d}'.format(alldays.n_obs))
alldays = alldays[alldays.obs['mt_frac'] < 0.2]
print('Number of cells after MT filter: {:d}'.format(alldays.n_obs))

sc.pp.filter_cells(alldays, min_genes = 1200)
print('Number of cells after gene filter: {:d}'.format(alldays.n_obs))
#Filter genes:
print('Total number of genes: {:d}'.format(alldays.n_vars))

# Min 20 cells - filters out 0 count genes
sc.pp.filter_genes(alldays, min_cells=20)
print('Number of genes after cell filter: {:d}'.format(alldays.n_vars))

## Normalization

In [ ]:
#Keep a copy of the raw, filtered data in a separate anndata object
alldays_counts = alldays.copy()
#Log-transform data and perform quantile normalisation
sc.pp.log1p(alldays)
alldays.X = sci.sparse.csr_matrix(scanpy.preprocessing.simple.normalize_per_cell_weinreb16_deprecated(alldays.X.toarray(), max_fraction=0.05, mult_with_mean=True))
alldays.raw = alldays

## Filter highly variable genes

In [ ]:
# Keep a list of highly variable genes
disp_filter = sc.pp.filter_genes_dispersion(alldays.X, flavor='cell_ranger', n_top_genes=4000, log=False, )
print('Number of highly variable genes: {:d}'.format(alldays[:, disp_filter['gene_subset']].n_vars))
alldays.var['highly_variable_genes'] = disp_filter['gene_subset']
alldays.var['expression_mean'] = disp_filter['means']
alldays.var['dispersion'] = disp_filter['dispersions']
sc.pl.filter_genes_dispersion(disp_filter)

Create a data set with only the highly variable genes

In [ ]:
alldays_hvg = alldays.copy()
alldays_hvg = alldays_hvg[:, disp_filter['gene_subset']]

# Read and write data

In [701]:
alldays_hvg.write('/adata_hvg_nonfiltered.h5ad')

In [ ]:
alldays.write('/adata_nonfiltered.h5ad')

In [649]:
alldays_hvg=sc.read('./adata_hvg_nonfiltered.h5ad')

In [ ]:
alldays=sc.read('./adata_nonfiltered.h5ad')